In [ ]:
from setup_notebook import set_path_to_root

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from data.database import GrandExchangeDB

In [ ]:
ge = GrandExchangeDB()

Select items which have on average more than 1m sold each hour

In [ ]:
ge.query_db("""
SELECT *
FROM PRICES
LIMIT 1000
""")

In [ ]:
def load_price_data():
    prices = ge.query_db(
        """
        WITH ITEMS_TRADED AS (
        SELECT item_id
                ,AVG(avgHighPrice * highPriceVolume) as amnt_traded_high
        FROM PRICES
        WHERE avgHighPrice IS NOT NULL
        AND highPriceVolume IS NOT NULL
        GROUP BY item_id
        )
        SELECT PRICES.*
            ,ITEMS.name
        FROM PRICES 
        INNER JOIN (
            SELECT DISTINCT item_id
            FROM ITEMS_TRADED
            WHERE amnt_traded_high > 1e6
        ) as SELECTED_ITEMS
        ON PRICES.item_id = SELECTED_ITEMS.item_id
        LEFT JOIN ITEMS
        ON ITEMS.id = PRICES.item_id
        """
    ).drop('index', axis=1)

    # Keep only items which have records for 95% of the selected time period
    n_periods = prices['datetime'].nunique()
    df = prices[['item_id']].value_counts().reset_index(name='count')
    df = df.query(f'count >= {round(n_periods * 0.95)}')
    
    prices = prices[prices['item_id'].isin(df['item_id'])].copy()
    prices['item_id'] = prices['item_id'].astype(int)
    prices['datetime'] = pd.to_datetime(prices['datetime'])

    # Add single values for price and volume
    prices['price'] = (prices['avgLowPrice'] + prices['avgHighPrice']) / 2
    prices['margin'] = prices['avgHighPrice'] - prices['avgLowPrice']
    prices['volume'] = prices['highPriceVolume'] + prices['lowPriceVolume']
    prices.drop(['avgHighPrice', 'avgLowPrice', 'highPriceVolume', 'lowPriceVolume'], axis=1, inplace=True)

    return prices

prices = load_price_data()

In [ ]:
prices.head()

In [ ]:
item_key = prices[['item_id', 'name', 'price', 'volume']].copy()
item_key['amount'] = item_key['volume'] * item_key['price']
item_key = item_key.groupby(['item_id', 'name'])['amount'].sum().reset_index()

In [ ]:
def plot_single_item_prices(item_id):
    df = prices[prices['item_id'] == item_id].copy()
    item_name = df['name'].unique()[0] + ' prices'
    sns.set(rc={'figure.figsize':(15, 7)})
    ax = sns.lineplot(data=df, x='datetime', y='price', color='blue', alpha=0.5)
    ax.set_title(item_name)
    plt.fill_between(df.datetime, df['price'] - df['margin'], df['price'] + df['margin'], color='grey', alpha=0.5)
    plt.show()

In [ ]:
plot_single_item_prices(21003)

### Detect and remove outliersplot_single_item_prices(2363)

In [ ]:
plot_single_item_prices(2363)

In [ ]:
df = prices[prices['item_id'] == 2363].copy()
df['price_change'] = df.sort_values('datetime')['price'].diff()
df.loc[df['price_change'].isna(), 'price_change'] = 0

In [ ]:
sns.histplot(df['price_change'])

Difference is normally distributed - use z scores to remove outliers

**Add a single measure for price and volume traded**
- **Price**: take the average of high and low price
- **Volume**: Take the sum of high/low traded volume
- **Margin**: the difference between high and low prices at that timestep

Can we detect any time-series outliers?

### Analysis

1. How many items?
1. Distribution of average prices (all items)
1. Distribution of variability (stddev)
1. Plots of margin against price over time
1. Ratio of price to amount traded (scatter)


### DTW find similar clusters of items

### Add useful stock feature

### Test multivariate forecasting?